In [1]:
import json
import pickle
import itertools
from pathlib import Path
from contextlib import redirect_stdout

import numpy as np
from pycocotools.cocoeval import COCOeval
from tqdm.notebook import tqdm

from prettytable import PrettyTable

In [2]:
ROOT = Path("./").resolve()

DETECTRON = ROOT / "detectron"
# https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md
BASELINES_IN_PRE = {
    "mask_rcnn_R_50_C4_1x":   [36.8, 32.2],
    "mask_rcnn_R_50_DC5_1x":  [38.3, 34.2],
    "mask_rcnn_R_50_FPN_1x":  [38.6, 35.2],
    "mask_rcnn_R_50_C4_3x":   [39.8, 34.4],
    "mask_rcnn_R_50_DC5_3x":  [40.0, 35.9],
    "mask_rcnn_R_50_FPN_3x":  [41.0, 37.2],
    "mask_rcnn_R_101_C4_3x":  [42.6, 36.7],
    "mask_rcnn_R_101_DC5_3x": [41.9, 37.3],
    "mask_rcnn_R_101_FPN_3x": [42.9, 38.6],
    "mask_rcnn_X_101_32x8d_FPN_3x": [44.3, 39.5]
    }
BASELINES_NO_PRE = {
    "mask_rcnn_R_50_FPN_100ep_LSJ":  [44.6, 40.3],
    "mask_rcnn_R_50_FPN_200ep_LSJ":  [46.3, 41.7],
    "mask_rcnn_R_50_FPN_400ep_LSJ":  [47.4, 42.5],
    "mask_rcnn_R_101_FPN_100ep_LSJ": [46.4, 41.6],
    "mask_rcnn_R_101_FPN_200ep_LSJ": [48.0, 43.1],
    "mask_rcnn_R_101_FPN_400ep_LSJ": [48.9, 43.7],
    "mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ": [46.0, 41.3],
    "mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ": [48.1, 43.1],
    "mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ": [48.6, 43.5],
    "mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ": [46.1, 41.6],
    "mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ": [47.8, 43.0],
    "mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ": [48.2, 43.3]
    }
OTHER_SETTINGS = {
    "cascade_mask_rcnn_R_50_FPN_1x": [42.1, 36.4],
    "cascade_mask_rcnn_R_50_FPN_3x": [44.3, 38.5],
    "mask_rcnn_R_50_FPN_1x_dconv_c3-c5": [41.5, 37.5],
    "mask_rcnn_R_50_FPN_3x_dconv_c3-c5": [42.7, 38.5],
    "panoptic_fpn_R_101_dconv_cascade_gn_3x": [47.4, 41.3],
    "cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv": [50.2, 44.0]
    }
# https://github.com/HobbitLong/PyContrast/tree/master/pycontrast/detection
SELF_SUPERVISED = {
    "R_50_FPN_Random_6x":     [42.7, 38.6],
    "R_50_FPN_Supervised_6x": [42.6, 38.5],
    "R_50_FPN_InstDis_1x":    [38.8, 35.2],
    "R_50_FPN_InstDis_2x":    [41.3, 37.3],
    "R_50_FPN_PIRL_1x":       [38.6, 35.1],
    "R_50_FPN_PIRL_2x":       [41.2, 37.4],
    "R_50_FPN_MoCo_v1_1x":    [39.4, 35.6],
    "R_50_FPN_MoCo_v1_2x":    [41.7, 37.5],
    "R_50_FPN_MoCo_v2_2x":    [41.7, 37.6],
    "R_50_FPN_InfoMin_1x":    [40.6, 36.7],
    "R_50_FPN_InfoMin_2x":    [42.5, 38.4],
    "R_50_FPN_InfoMin_6x":    [43.6, 39.2]
    }

# https://gist.github.com/szagoruyko/9c9ebb8455610958f7deaa27845d7918
DETR = ROOT / "detr"
DETR_PANOPTIC = {
    "detr_r50":      [38.8, 31.1],
    "detr_r50_dc5": [40.2, 31.9],
    "detr_r101":     [40.1, 33.0]
}

# https://github.com/AlexeyAB/darknet
# no scores reported for val2017, we checked test-dev with coco server though
DARKNET = ROOT / "darknet"
YOLO = {
    "yolov3" :    [-1, -1],
    "yolov4" :    [-1, -1],
    "yolov4-csp": [-1, -1],
    "yolov4-p5":  [-1, -1],
    "yolov4-p6":  [-1, -1]
}

# https://github.com/SwinTransformer/Swin-Transformer-Object-Detection
SWIN = ROOT / "swin"
SWIN_MODELS = {
    "mask_rcnn_swin-T_1x" : [43.7, 39.8],
    "mask_rcnn_swin-T_3x" : [46.0, 41.6],
    "mask_rcnn_swin-S_3x" : [48.5, 43.3],  
    "cascade_mask_rcnn_swin-T_1x" : [48.1, 41.7],
    "cascade_mask_rcnn_swin-T_3x" : [50.4, 43.7],  
    "cascade_mask_rcnn_swin-S_3x" : [52.0, 45.0],
    "cascade_mask_rcnn_swin-B_3x" : [51.9, 45.0]
}

# https://github.com/easton-cau/SOTR
# no scores reported for val2017, we checked test-dev with coco server though
SOTR = ROOT / "sotr"
SOTR_MODELS = {
    "sotr_R101" :    [-1, -1],
    "sotr_R101_dcn": [-1, -1]
}

# https://github.com/aim-uofa/AdelaiDet/tree/master/configs/SOLOv2
# no scores reported for val2017, we checked test-dev with coco server though
SOLO = ROOT / "solov2"
SOLO_MODELS = {
    "solov2_R50_3x" : [-1, -1],
    "solov2_R101_3x": [-1, -1]
}

# https://github.com/dbolya/yolact
# no scores reported for val2017, we checked test-dev with coco server though
YOLACT = ROOT / "yolact"
YOLACT_MODELS = {
    "yolact-R50-FPN" :  [-1, -1],
    "yolact-R101-FPN" : [-1, -1],
    "yolact-plus-R50-FPN":  [-1, -1],
    "yolact-plus-R101-FPN": [-1, -1]
}

BMASK = ROOT / "bmask"
BMASK_MODELS = {
    "bmask_rcnn_r50_1x" : [-1, 36.6],
    "bmask_rcnn_r101_1x": [-1, 38.0],
    "cascade_bmask_rcnn_r50_1x" : [-1, 37.5],
    "cascade_bmask_rcnn_r101_1x" : [-1, 39.1]
}

BCNET = ROOT / "bcnet"
BCNET_MODELS = {
    "fcos_imprv_R_101_FPN" : [-1, 39.6]
}

In [3]:
# COCO EVAL format https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/cocoeval.py

# precision  - [TxRxKxAxM] 
# recall     - [TxKxAxM]
# T: thresholds
# R: recall thresholds
# K: categories
# A: areas 
# M: max detections

def printCocoEvalParams(eval_object):
    params = eval_object['params']
    print(f"IOU Type: {params.iouType}")
    print(f"Counts: {eval_object['counts']}")      
    print(f"T ( {len(params.iouThrs)}): {params.iouThrs}")
    print(f"R ({len(params.recThrs)}): {min(params.recThrs)}, ..., {max(params.recThrs)}")
    print(f"K ( {len(params.catIds)}): {min(params.catIds)}, ..., {max(params.catIds)} ")
    print(f"A (  {len(params.areaRngLbl)}): {params.areaRngLbl}")
    print(f"M (  {len(params.maxDets)}): {params.maxDets}")

# custom summarize method:
# runs summarize() on a loaded eval dict {} and returns stats instead of printing them
def COCOeval_summarize(loaded_eval_dict):
    eval_proxy = COCOeval()
    eval_proxy.eval   = loaded_eval_dict
    eval_proxy.params = loaded_eval_dict['params']
    eval_proxy.summarize()
    return eval_proxy.stats

## Load detection results

In [4]:
COCO = "coco/val2017"
STYLIZED_COCO            = "stylized_coco"
STYLIZED_COCO_OBJECTS    = "stylized_coco_objects"
STYLIZED_COCO_BACKGROUND = "stylized_coco_background"
STYLE_VERSIONS = ["1"]

ALPHAS_fs   = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ALPHAS_ps   =      [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]      # 0.0=coco, 1.0 same as fs 1.0
BLEND_MODES = {"pixel_space": ALPHAS_ps, "feature_space": ALPHAS_fs}

EVAL_BBOX  = "coco_eval_bbox.pkl"          # coco eval object 'bbox'
EVAL_SEGM  = "coco_eval_segm.pkl"          # coco eval object 'segm'
IOU_TYPES  = {'bbox': EVAL_BBOX, 'segm': EVAL_SEGM}

In [5]:
MODELS = {}
MODELS.update(BASELINES_IN_PRE)
MODELS.update(BASELINES_NO_PRE)
MODELS.update(SELF_SUPERVISED)
MODELS.update(OTHER_SETTINGS)
MODELS.update(DETR_PANOPTIC)
MODELS.update(YOLO)
MODELS.update(SWIN_MODELS)
MODELS.update(SOTR_MODELS)
MODELS.update(SOLO_MODELS)
MODELS.update(YOLACT_MODELS)
MODELS.update(BMASK_MODELS)
MODELS.update(BCNET_MODELS)
print(len(MODELS))

68


In [6]:
RESULTS = {}
MODELS = {}
MODELS.update(BASELINES_IN_PRE)
MODELS.update(BASELINES_NO_PRE)
MODELS.update(SELF_SUPERVISED)
MODELS.update(OTHER_SETTINGS)
MODELS.update(DETR_PANOPTIC)
MODELS.update(YOLO)
MODELS.update(SWIN_MODELS)
MODELS.update(SOTR_MODELS)
MODELS.update(SOLO_MODELS)
MODELS.update(YOLACT_MODELS)
MODELS.update(BMASK_MODELS)
MODELS.update(BCNET_MODELS)

# load results
for model in tqdm(list(MODELS.keys())):    
    RESULTS[model] = {}

    # add results from coco
    RESULTS[model]['coco'] = {}
    for iou_type, eval_file in IOU_TYPES.items():
        if iou_type == 'segm' and model in YOLO.keys(): continue
        # set path
        if 'detr' in model:                 PATH = DETR
        elif 'yolo' in model:               PATH = DARKNET
        elif model in SWIN_MODELS.keys():   PATH = SWIN
        elif model in SOTR_MODELS.keys():   PATH = SOTR
        elif model in SOLO_MODELS.keys():   PATH = SOLO
        elif model in YOLACT_MODELS.keys(): PATH = YOLACT
        elif model in BMASK_MODELS.keys():  PATH = BMASK
        elif model in BCNET_MODELS.keys():  PATH = BCNET
        else: 
            PATH = DETECTRON

        with open(PATH / model / COCO / eval_file, 'rb') as f:
            eval_dict = pickle.load(f)
            with redirect_stdout(None): # printing is slow
                RESULTS[model]['coco'][iou_type] = COCOeval_summarize(eval_dict)

    # add results from stylized coco, objects, background
    for dataset in [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]:
        RESULTS[model][dataset] = {}
        for style in STYLE_VERSIONS: # 1
            RESULTS[model][dataset][style] = {}
            for mode, alpha_values in BLEND_MODES.items(): # feature space, pixel_space
                RESULTS[model][dataset][style][mode] = {}
                for alpha in alpha_values:
                    RESULTS[model][dataset][style][mode][alpha] = {}
                    for iou_type, eval_file in IOU_TYPES.items():
                        if iou_type == 'segm' and model in YOLO.keys(): continue
                        if 'detr' in model:                 PATH = DETR
                        elif 'yolo' in model:               PATH = DARKNET
                        elif model in SWIN_MODELS.keys():   PATH = SWIN
                        elif model in SOTR_MODELS.keys():   PATH = SOTR
                        elif model in SOLO_MODELS.keys():   PATH = SOLO
                        elif model in YOLACT_MODELS.keys(): PATH = YOLACT
                        elif model in BMASK_MODELS.keys():  PATH = BMASK
                        elif model in BCNET_MODELS.keys():  PATH = BCNET
                        else:
                            PATH = DETECTRON
                        with open(PATH / model / dataset / style / mode / str(alpha) / eval_file, 'rb') as f:
                            eval_dict = pickle.load(f)
                            with redirect_stdout(None): # printing is slow
                                RESULTS[model][dataset][style][mode][alpha][iou_type] = COCOeval_summarize(eval_dict)


  0%|          | 0/68 [00:00<?, ?it/s]

In [7]:
STATS = {
    "AP"   :  0, #  0: Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]
    "AP50" :  1, #  1: Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]
    "AP75" :  2, #  2: Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]
    "APs"  :  3, #  3: Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]
    "APm"  :  4, #  4: Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]
    "APl"  :  5, #  5: Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]
    "AR1"  :  6, #  6: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]
    "AR10" :  7, #  7: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]
    "AR100":  8, #  8: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]
    "ARs"  :  9, #  9: Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]
    "ARm"  : 10, # 10: Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]
    "ARl"  : 11  # 11: Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]
}

# sort by statistic for easy plotting, alphas are concatenated
SUMMARY = {}
for model in list(MODELS.keys()):    
    SUMMARY[model] = {}
    for dataset in [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]:
        SUMMARY[model][dataset] = {}
        for style in STYLE_VERSIONS: # 1
            SUMMARY[model][dataset][style] = {}
            for mode, alpha_values in BLEND_MODES.items(): # feature space, pixel_space
                SUMMARY[model][dataset][style][mode] = {}
                for iou_type in IOU_TYPES.keys():
                    if iou_type == 'segm' and model in YOLO.keys(): continue
                    SUMMARY[model][dataset][style][mode][iou_type] = {}

                    # new sort by statistics now
                    for stat_key, stat_idx in STATS.items():
                        SUMMARY[model][dataset][style][mode][iou_type][stat_key] = []
                        if mode == 'feature_space':
                            for alpha in alpha_values:
                                s = RESULTS[model][dataset][style][mode][alpha][iou_type][stat_idx]
                                SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)
                        if mode == 'pixel_space':    
                            # add coco (alpha=0 in pixel space)
                            s = RESULTS[model]['coco'][iou_type][stat_idx]
                            SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)

                            # add [0.1, ..., 0.9]
                            for alpha in alpha_values:
                                s = RESULTS[model][dataset][style][mode][alpha][iou_type][stat_idx]
                                SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)

                            # add stylized coco, object, background (alpha=1 to pixel space list)
                            s = RESULTS[model][dataset][style]['feature_space'][1.0][iou_type][stat_idx]
                            SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)


In [8]:
def compare_models(models, DATASET, VERSION='1', 
                   BLEND_MODE='pixel_space', 
                   IOU_TYPE='segm', 
                   ALPHA=0.0,
                   STATS=["AP", "AP50", "AP75", "APs", "APm", "APl"],
                   #STATS=["AR1", "AR10", "AR100", "ARs", "ARm", "ARl"],
                   REL_ERROR=False):

            a_idx = ALPHAS_fs.index(ALPHA)
            
            if ALPHA == 0.0 and BLEND_MODE == "pixel_space":
                headers = [f"Models ({BLEND_MODE},{IOU_TYPE} @ {ALPHA})"] + ["Repo.AP"] + STATS
            else:
                headers = [f"Models ({BLEND_MODE},{IOU_TYPE} @ {ALPHA})"] + STATS
                    
            t = PrettyTable(headers)
            t.align[f"Models ({BLEND_MODE},{IOU_TYPE} @ {ALPHA})"] = 'l'

            for model in models:
                if IOU_TYPE == 'segm' and model in YOLO.keys(): continue
                row = [model]
                # reported Performance
                if ALPHA == 0.0 and BLEND_MODE == "pixel_space":
                    row.append(MODELS[model][0 if IOU_TYPE == 'bbox' else 1])
           
                for stat in STATS:
                    y = SUMMARY[model][DATASET][VERSION][BLEND_MODE][IOU_TYPE][stat][a_idx]
                
                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[model][DATASET][VERSION]['pixel_space'][IOU_TYPE][stat][0]
                        y = y / coco_score

                    row.append(np.round(y*100,1))
                t.add_row(row)

            print(t)

In [ ]:
MODELS = {}
MODELS.update(BASELINES_IN_PRE)
MODELS.update(BASELINES_NO_PRE)
MODELS.update(SELF_SUPERVISED)
MODELS.update(OTHER_SETTINGS)
MODELS.update(DETR_PANOPTIC)
MODELS.update(YOLO)
MODELS.update(SWIN_MODELS)
MODELS.update(SOTR_MODELS)
MODELS.update(SOLO_MODELS)
MODELS.update(YOLACT_MODELS)
MODELS.update(BMASK_MODELS)
MODELS.update(BCNET_MODELS)


In [28]:
custom_list = [
    "mask_rcnn_R_50_FPN_3x",
    "mask_rcnn_R_50_FPN_3x_dconv_c3-c5"
]

In [34]:
#compare_models(list(MODELS.keys()), STYLIZED_COCO_BACKGROUND,
compare_models(custom_list, STYLIZED_COCO_OBJECTS,
               BLEND_MODE='feature_space',
               #BLEND_MODE='pixel_space',
               IOU_TYPE='segm',
               ALPHA=1.0,
               REL_ERROR=True)

+-----------------------------------+------+------+------+------+------+------+
| Models (feature_space,segm @ 1.0) |  AP  | AP50 | AP75 | APs  | APm  | APl  |
+-----------------------------------+------+------+------+------+------+------+
| mask_rcnn_R_50_FPN_3x             | 40.5 | 40.7 | 39.9 | 30.1 | 39.5 | 47.5 |
| mask_rcnn_R_50_FPN_3x_dconv_c3-c5 | 48.5 | 48.2 | 47.8 | 34.7 | 46.7 | 55.2 |
+-----------------------------------+------+------+------+------+------+------+
